In [12]:
import pandas as pd
import numpy as np
import os
import h5py

In [13]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install osmnx

Note: you may need to restart the kernel to use updated packages.


In [34]:
# geography

import geopandas as gpd
import osmnx as ox
import csv
import ast
from geopy.geocoders import Nominatim
from geopy.point import Point

In [49]:
df = pd.read_csv('/Users/katiecason/Desktop/207_folder/w207_finalproject/tweets_slightly_cleaned.csv')

/Users/katiecason/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [50]:
df = df[df['geo'].notna()]

In [51]:
df.geo = df.geo.apply(lambda x: ast.literal_eval(x))

In [52]:
df.geo = df.geo.apply(lambda x: x['coordinates'])

In [53]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [54]:
#manually filtering latitude and longitude to find locations

#df = df[df['coordinates'].notna()]
#data = df['coordinates'].str.split(',', n=1, expand = True)
#df['longitude'] = data[0]
#df['latitude'] = data[1]
#df['coordinates'] = df[['latitude', 'longitude']].apply(lambda x: ','.join(x), axis=1)

In [55]:
df.created_at = df.created_at.apply(lambda x: pd.to_datetime(x).date())

In [56]:
date_after = pd.to_datetime('2020-11-21') #this is where our twitter data begins
date_before = pd.to_datetime('2020-12-20') #this is where our twitter data ends
df = df[df['created_at'] >= date_after]
df = df[df['created_at'] <= date_before]

In [57]:
df['longitude'] = df['geo'].apply(lambda x: x[1])
df['latitude'] = df['geo'].apply(lambda x: x[0])
df = df[df['longitude'].between(-125, -115)]
df = df[df['latitude'].between(33,42)]
df

,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang,longitude,latitude
245611,2020-11-21,Phaedra Mae is ready for a very chill weekend ...,"[34.07375505, -118.33470047]","{'id': '3b77caf94bfc81fe', 'url': 'https://api...",False,"[0, 131]","{'id': 15573871, 'id_str': '15573871', 'name':...",1329940949762519040,"{'hashtags': [{'text': 'phaedramaegolden', 'in...",True,en,-118.334700,34.073755
245616,2020-11-21,Sigh. I thought we’d gotten past shortages in ...,"[37.898565, -122.32041742]","{'id': '029dd1f28dbd3d26', 'url': 'https://api...",False,"[0, 123]","{'id': 12091872, 'id_str': '12091872', 'name':...",1329946069346766848,"{'hashtags': [{'text': 'covid19', 'indices': [...",False,en,-122.320417,37.898565
245618,2020-11-21,"Just posted a photo @ Corona, California https...","[33.8753, -117.566]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 64]","{'id': 955459485002354691, 'id_str': '95545948...",1329946405109174273,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-117.566000,33.875300
245621,2020-11-21,I was on Pacific Union College campus providin...,"[38.57040968, -122.44076793]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",False,"[0, 173]","{'id': 3110772967, 'id_str': '3110772967', 'na...",1329947200265347072,"{'hashtags': [{'text': 'collegestudent', 'indi...",False,en,-122.440768,38.570410
245627,2020-11-21,"Just posted a photo @ Corona, California https...","[33.8753, -117.566]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 64]","{'id': 72482305, 'id_str': '72482305', 'name':...",1329953554455429120,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-117.566000,33.875300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251365,2020-12-01,In times of quarantine. All you need after cur...,"[37.7793, -122.419]","{'id': '5a110d312052166f', 'url': 'https://api...",False,"[0, 219]","{'id': 18465677, 'id_str': '18465677', 'name':...",1333601950525509632,"{'hashtags': [{'text': 'sfcurfew', 'indices': ...",False,en,-122.419000,37.779300
251377,2020-12-01,"Just posted a photo @ Corona, California https...","[33.8753, -117.566]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 64]","{'id': 912199619685244929, 'id_str': '91219961...",1333614607043334146,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-117.566000,33.875300
251379,2020-12-01,10 High Paying Online Jobs You Can Do Working ...,"[33.7174708, -117.8311428]","{'id': '02215ae29a77567c', 'url': 'https://api...",False,"[0, 100]","{'id': 4804223635, 'id_str': '4804223635', 'na...",1333615887438450689,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-117.831143,33.717471
251385,2020-12-01,10 High Paying Online Jobs You Can Do Working ...,"[33.7174708, -117.8311428]","{'id': '02215ae29a77567c', 'url': 'https://api...",False,"[0, 100]","{'id': 4804223635, 'id_str': '4804223635', 'na...",1333618726797602816,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-117.831143,33.717471


In [58]:
geolocator = Nominatim(user_agent='katiecason@berkeley.edu')
location = df['geo'].apply(lambda x: geolocator.reverse(x))

In [59]:
states = location.apply(lambda x: x.raw['address'].get('state', ''))
counties = location.apply(lambda x: x.raw['address'].get('county', ''))

In [60]:
df['state'] = states
df['county'] = counties

In [61]:
df = df[df.state == 'California']

In [62]:
df.to_csv('../data/november-december.csv')